In [1]:
import sys  
sys.path.insert(0, '../search-engine')
from utils import *
import pandas as pd

In [3]:
# cridar tf-idf --> 
data = get_tweets(None, None, mode = "read", data_directory = '../data/')
data = data[:10000]

tweetsDict = create_tweets_dict(data)
index, tf, df, idf = create_index_tfidf(data, len(data))

In [4]:
query = "trump"
scores = personalized_rank(query, index, tf, idf, tweetsDict, data)

scores

,tf-idf,partial,personalized,w2v
1335568907311206400,0.432033,0.000111,0.432083,40.779912
1335568907411869698,0.379019,0.004974,0.381285,69.648305
1335568908481335297,0.379019,0.000005,0.379021,61.629453
1335568908812759040,0.394459,0.000073,0.394492,81.121578
1335568909072785411,0.507454,0.000009,0.507458,47.732741
...,...,...,...,...
1335563686803542018,0.411948,0.002139,0.412922,59.468572
1335563687311060999,0.483133,0.000245,0.483245,71.810061
1335563687453712390,0.411948,0.001637,0.412693,68.387467
1335563687935897602,0.411948,0.000645,0.412242,55.762444


In [43]:
# Import performed clustering in RQ1
cluster_df = pd.read_csv("../other-outputs/clustering_output_10000.csv", index_col = "Unnamed: 0")
scores.index = scores.index.astype('int64')
cluster_df.head()

,Text,Cluster
1335568906937896960,RT @9NEWSNANCY: 🤬@SpeakerPelosi has a LOT of d...,2
1335568907059466241,The Canadian strikes again https://t.co/Wj2i8L...,2
1335568907059535874,Call your legislators! #FightForTrump https://...,0
1335568907139231744,RT @BernardKerik: These are the four cowardice...,2
1335568907311132674,RT @realDonaldTrump: The answer to the Democra...,0


In [49]:
scores = scores.join(cluster_df, how="left")
scores.head()

,tf-idf,partial,personalized,w2v,Text,Cluster
1335568907311206400,0.432033,0.000111,0.432083,40.779912,@SoCal_For_Life @DanScavino That’s disgusting....,0
1335568907411869698,0.379019,0.004974,0.381285,69.648305,RT @smalltownandrew: Donald Trump Demands Vote...,0
1335568908481335297,0.379019,0.000005,0.379021,61.629453,RT @mmpadellan: trump just told his Valdosta c...,0
1335568908812759040,0.394459,0.000073,0.394492,81.121578,RT @realDonaldTrump: ***Live Updates*** Trump ...,0
1335568909072785411,0.507454,0.000009,0.507458,47.732741,@petercbarbour @JennaEllisEsq No. That 2 minut...,2


In [53]:
clusters_size = scores["Cluster"].value_counts()

In [61]:
div_score = []
lamb = 1/3

max_tfidf = scores["tf-idf"].max(axis=0)
for i, value in scores.iterrows():
    partial = 1 - clusters_size[value["Cluster"]]/len(scores)
    score = value["tf-idf"] + lamb * max_tfidf * partial
    div_score.append(score)

In [62]:
scores["diversity"] = div_score

In [65]:
scores_reduced = scores[["tf-idf", "diversity", "Cluster"]]
scores_reduced

,tf-idf,diversity,Cluster
1335568907311206400,0.432033,0.550327,0
1335568907411869698,0.379019,0.497314,0
1335568908481335297,0.379019,0.497314,0
1335568908812759040,0.394459,0.512753,0
1335568909072785411,0.507454,0.844602,2
...,...,...,...
1335563686803542018,0.411948,0.530242,0
1335563687311060999,0.483133,0.601428,0
1335563687453712390,0.411948,0.530242,0
1335563687935897602,0.411948,0.530242,0


In [67]:
display_ranking(scores_reduced, tweetsDict, query, score = "tf-idf", top_k = 10)


Top-10 documents sorted by tf-idf score
1335559283564453889: 🇺🇸trump https://t.co/P3ctsnhebv

1335565393147080710: *with Trump

1335568968795516928: RT @BeNosey: UK HERE, I TOTALLY GET TRUMP #TRUMP https://t.co/COZIUWFFn6

1335568930216300544: RT @JoyAnnReid: Nixon was the Trump before Trump.

1335567431499051010: @ANINewsUP @ANI If they are farmer than i am #Trump

1335571192569270274: @WarrenWhitlock Except #Trump maybe ? :-)

1335571398287368192: #Trump is such a #Creep

1335567817530171392: RT @Pollyanna1698: Trump Won!!! https://t.co/EFuNGXJhRb

1335572554698911744: Trump card https://t.co/JpF6FuaiS8

1335559313788727302: Trump Nation! https://t.co/Akjs7PFhRW



In [68]:
display_ranking(scores_reduced, tweetsDict, query, score = "diversity", top_k = 10)


Top-10 documents sorted by diversity score
1335559283564453889: 🇺🇸trump https://t.co/P3ctsnhebv

1335565393147080710: *with Trump

1335567431499051010: @ANINewsUP @ANI If they are farmer than i am #Trump

1335572554615033857: @TheDemCoalition @BuzzFeedNews How about #Trump revealing his taxes?

1335568968795516928: RT @BeNosey: UK HERE, I TOTALLY GET TRUMP #TRUMP https://t.co/COZIUWFFn6

1335568930216300544: RT @JoyAnnReid: Nixon was the Trump before Trump.

1335559344113520640: RT @BeNosey: Numbers. 17 / 9+1+7 17=Q. #TRUMP https://t.co/nkK6aoUBLP https://t.co/b1VMhnk0gy https://t.co/zE4DA0hCM8 https://t.co/HWC8s…

1335571192569270274: @WarrenWhitlock Except #Trump maybe ? :-)

1335571458194542593: A trump fart in action. https://t.co/qomCcGqdCM

1335571179000754177: @ScottFishman TRUMP



In [75]:
## PROVES RANKING CORRELATION
from scipy.stats import spearmanr
coef, p = spearmanr(scores["tf-idf"], scores["diversity"])
coef

0.6606093966756547

In [79]:
print(scores_reduced.sort_values(by="diversity", ascending = False)[:20]["Cluster"].value_counts())
print(scores_reduced.sort_values(by="tf-idf", ascending = False)[:20]["Cluster"].value_counts())

0    16
2     4
Name: Cluster, dtype: int64
0    18
2     2
Name: Cluster, dtype: int64
